<a href="https://colab.research.google.com/github/opailo/dv_psych_project/blob/main/main_script/Psych_Project_Final_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import PIL.Image, PIL.ImageFont, PIL.ImageDraw
import os
from random import shuffle
import csv
import pandas as pd 
from itertools import cycle

# Key for file names 

* idx 0 = Race + Gender + Expressor Number 
* idx 1 = irrelevant for project
* idx 2 = Posture
* idx 3 = Gender
* idx 4 = Scene

In [ ]:
directory = 'C:/Users/Otavio/Videos/FPCFull/'
fd = 'C:/Users/Otavio/Videos/final_directory/'

fpc_combos_list = ['Disgust_Anger_Sadness',
                   'Anger_Disgust_Sadness',
                   'Sadness_Fear_Hoy',
                   'Fear_Anger_Disgust',
                   'Hoy_Sadness_Fear',
                   'Neutral_Disgust_Anger',
                   'Neutral_Disgust_Hoy',
                   'Neutral_Disgust_Sadness',
                   'Neutral_Hoy_Anger',
                   'Disgust_Anger_Anger',
                   'Anger_Disgust_Disgust',
                   'Anger_Fear_Hoy',
                   'Anger_Fear_Sadness',
                   'Sadness_Fear_Neutral',
                   'Disgust_Disgust_Disgust',
                   'Hoy_Hoy_Hoy',
                   'Fear_Fear_Fear',
                   'Anger_Anger_Anger',
                   'Sadness_Sadness_Sadness',
                   'Neutral_Neutral_Neutral']



In [ ]:
def split_fpc_combos(fpc_combos_list):
  ''' 
  Splits the list of fpc combos by underscore and returns a list containing the split fpc's
  '''
  split_fpc_combos_list = []

  for fpc in fpc_combos_list:
    split_fpc_combos_list.append(fpc.split('_'))

  return split_fpc_combos_list

In [ ]:
def display_image(image_path):
  """
  Function for displaying image 
  """
  
  displayed_image = PIL.Image.open(image_path)

  return displayed_image 
  # return displayed_image.show()

In [ ]:
# Places image paths into a list and return
def file_path_list_generator(directory):
  """
  This is going to place the full image paths into one list and 
  place the the individual file names into a seperate list 
  """
  file_paths_list = []
  file_names_list = []

  for filename in os.listdir(directory):
      f = os.path.join(directory, filename)

      if os.path.isfile(f):
        file_paths_list.append(f)
        file_names_list.append(filename)

  return file_paths_list, file_names_list

In [ ]:
def split_file_names(file_names_list):
  """
  This is going to split the file names into it's components and place them in individual lists 
  which will then be appended to new_list
  So file 'AM03_FO_Anger_Male_Joy5.jpg' will be ['AM03', 'FO', 'Anger', 'Male', 'Joy5']
  """

  split_file_names_list = []
  for filename in file_names_list:
    split_file_names_list.append(filename.replace('.jpg', '').split('_'))

  return split_file_names_list

In [ ]:
def create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race, participant_gender):
  '''
  This function creates the base dataset based on the participant's race and gender 

  Race is denoted by: A, B, H, W
  Gender is denoted by: M, F, O
  '''
  dataset_indexes = []
  pre_final_dataset = []

  # Loops through split_file_names_list and checks if Race and Gender matches 
  for filename in split_file_names_list: 
    if filename[0][0] == participant_race and filename[0][1] == participant_gender:
      # Appends the indices of the file_names that pass the conditions
      dataset_indexes.append(split_file_names_list.index(filename))

  # Shuffles the indices
  shuffle(dataset_indexes)
  shuffled_dataset_indexes = dataset_indexes

  i = 0

  while i < len(shuffled_dataset_indexes) - 1:
    # expressor condition, posture condition, scene condition
    if split_file_names_list[shuffled_dataset_indexes[i]][0][2] + split_file_names_list[shuffled_dataset_indexes[i]][0][3] != split_file_names_list[shuffled_dataset_indexes[i+1]][0][2] + split_file_names_list[shuffled_dataset_indexes[i+1]][0][3] and split_file_names_list[shuffled_dataset_indexes[i]][2] != split_file_names_list[shuffled_dataset_indexes[i+1]][2] and split_file_names_list[shuffled_dataset_indexes[i]][4] != split_file_names_list[shuffled_dataset_indexes[i+1]][4]:
      pre_final_dataset.append(file_paths_list[shuffled_dataset_indexes[i]])
    i+=1


  return pre_final_dataset

In [ ]:
def create_final_dataset(pre_final_dataset, fpc_combos_list):
  
  temp_filename_list = []
  final_dataset = []

  #Splits the fpc combos list by underscores into a new list
  split_fpc_combos_list = split_fpc_combos(fpc_combos_list)

  # Removes the directory from the filenames in the files in the pre_final_dataset
  # Remember that this is the shuffled dataset that has already been sorted for:
    # Race, gender, and makes sure that there aren't any duplicates in terms of posture, face, and backgrounds
  for filename in pre_final_dataset:
    temp_filename_list.append(filename.replace(directory, ""))

  # Splits the filenames by underscore
  split_temp_filename_list = split_file_names(temp_filename_list)

  # Loops through the fpc combos and checks them with their respective counterparts in the filenames
  # Adds file names that pass the conditions 
  #Conditions:
    # Condition 1: First letters of fpc for Face FPC must match
    # Condition 2: Posture FPC must match
    # Conditon 3: First letters for scene FPC must match
  for fpc in split_fpc_combos_list:
    for file_name in split_temp_filename_list:
      if fpc[0][0] == file_name[1][0] and fpc[1] == file_name[2] and fpc[2][0] == file_name[4][0]:
        final_dataset.append( pre_final_dataset[split_temp_filename_list.index(file_name)] )
        break
  return final_dataset

In [ ]:
def only_gender_is_other(directory, true_r):
      if true_r == 'A':
        #Creates first set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='A', participant_gender='M')

        final_dataset_1 = create_final_dataset(pre_final_dataset, fpc_combos_list)

        #Creates second set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='A', participant_gender='M')
        final_dataset_2 = create_final_dataset(pre_final_dataset, fpc_combos_list)

        #Creates third set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='A', participant_gender='M')
        final_dataset_3 = create_final_dataset(pre_final_dataset, fpc_combos_list)

        #Creates fourth set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='A', participant_gender='F')
        final_dataset_4 = create_final_dataset(pre_final_dataset, fpc_combos_list)

        #Creates fith set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='A', participant_gender='F')
        final_dataset_5 = create_final_dataset(pre_final_dataset, fpc_combos_list)

        #Creates sixth set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='A', participant_gender='F')
        final_dataset_6 = create_final_dataset(pre_final_dataset, fpc_combos_list)

        # Combines the final_datasets together
        full_dataset = final_dataset_1 + final_dataset_2 + final_dataset_3 + final_dataset_4 + final_dataset_5 + final_dataset_6

        # Shuffles the combined set
        shuffle(full_dataset)

        dataset = []

        # Loops through the full dataset and removes any duplicates and any excess files
        for file_paths in full_dataset:
          if file_paths not in dataset:
            dataset.append(file_paths)

        return dataset

      elif true_r == 'B':
        #Creates first set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='B', participant_gender='M')

        final_dataset_1 = create_final_dataset(pre_final_dataset, fpc_combos_list)

        #Creates second set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='B', participant_gender='M')
        final_dataset_2 = create_final_dataset(pre_final_dataset, fpc_combos_list)

        #Creates third set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='B', participant_gender='M')
        final_dataset_3 = create_final_dataset(pre_final_dataset, fpc_combos_list)

        #Creates fourth set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='B', participant_gender='F')
        final_dataset_4 = create_final_dataset(pre_final_dataset, fpc_combos_list)

        #Creates fith set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='B', participant_gender='F')
        final_dataset_5 = create_final_dataset(pre_final_dataset, fpc_combos_list)

        #Creates sixth set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='B', participant_gender='F')
        final_dataset_6 = create_final_dataset(pre_final_dataset, fpc_combos_list)

        # Combines the final_datasets together
        full_dataset = final_dataset_1 + final_dataset_2 + final_dataset_3 + final_dataset_4 + final_dataset_5 + final_dataset_6

        # Shuffles the combined set
        shuffle(full_dataset)

        dataset = []

        # Loops through the full dataset and removes any duplicates and any excess files
        for file_paths in full_dataset:
          if file_paths not in dataset:
            dataset.append(file_paths)

        return dataset

      elif true_r == 'H':
        #Creates first set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='H', participant_gender='M')
        final_dataset_1 = create_final_dataset(pre_final_dataset, fpc_combos_list)


        #Creates second set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='H', participant_gender='M')
        final_dataset_2 = create_final_dataset(pre_final_dataset, fpc_combos_list)

        #Creates third set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='H', participant_gender='M')
        final_dataset_3 = create_final_dataset(pre_final_dataset, fpc_combos_list)

        #Creates fourth set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='H', participant_gender='F')
        final_dataset_4 = create_final_dataset(pre_final_dataset, fpc_combos_list)

        #Creates fith set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='H', participant_gender='F')
        final_dataset_5 = create_final_dataset(pre_final_dataset, fpc_combos_list)

        #Creates sixth set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='H', participant_gender='F')
        final_dataset_6 = create_final_dataset(pre_final_dataset, fpc_combos_list)

        # Combines the final_datasets together
        full_dataset = final_dataset_1 + final_dataset_2 + final_dataset_3 + final_dataset_4 + final_dataset_5 + final_dataset_6

        # Shuffles the combined set
        shuffle(full_dataset)

        dataset = []

        # Loops through the full dataset and removes any duplicates and any excess files
        for file_paths in full_dataset:
          if file_paths not in dataset:
            dataset.append(file_paths)

        return dataset

      elif true_r == 'W':
        #Creates first set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='W', participant_gender='M')

        final_dataset_1 = create_final_dataset(pre_final_dataset, fpc_combos_list)

        #Creates second set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='W', participant_gender='M')
        final_dataset_2 = create_final_dataset(pre_final_dataset, fpc_combos_list)

        #Creates third set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='W', participant_gender='M')
        final_dataset_3 = create_final_dataset(pre_final_dataset, fpc_combos_list)

        #Creates fourth set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='W', participant_gender='F')
        final_dataset_4 = create_final_dataset(pre_final_dataset, fpc_combos_list)

        #Creates fith set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='W', participant_gender='F')
        final_dataset_5 = create_final_dataset(pre_final_dataset, fpc_combos_list)

        #Creates sixth set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='W', participant_gender='F')
        final_dataset_6 = create_final_dataset(pre_final_dataset, fpc_combos_list)

        # Combines the final_datasets together
        full_dataset = final_dataset_1 + final_dataset_2 + final_dataset_3 + final_dataset_4 + final_dataset_5 + final_dataset_6

        # Shuffles the combined set
        shuffle(full_dataset)

        dataset = []

        # Loops through the full dataset and removes any duplicates and any excess files
        for file_paths in full_dataset:
          if file_paths not in dataset:
            dataset.append(file_paths)

        return dataset


In [ ]:
def only_race_is_other(directory, true_g):
      if true_g == 'M':
        #Creates first set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='A', participant_gender='M')
        final_dataset_1 = create_final_dataset(pre_final_dataset, fpc_combos_list)

        #Creates second set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='B', participant_gender='M')
        final_dataset_2 = create_final_dataset(pre_final_dataset, fpc_combos_list)

        #Creates third set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='H', participant_gender='M')
        final_dataset_3 = create_final_dataset(pre_final_dataset, fpc_combos_list)

        #Creates fourth set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='W', participant_gender='M')
        final_dataset_4 = create_final_dataset(pre_final_dataset, fpc_combos_list)

        #Creates fifth set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='A', participant_gender='M')
        final_dataset_5 = create_final_dataset(pre_final_dataset, fpc_combos_list)

        #Creates sixth set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='B', participant_gender='M')
        final_dataset_6 = create_final_dataset(pre_final_dataset, fpc_combos_list)

        #Creates seventh set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='H', participant_gender='M')
        final_dataset_7 = create_final_dataset(pre_final_dataset, fpc_combos_list)

        #Creates eighth set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='W', participant_gender='M')
        final_dataset_8 = create_final_dataset(pre_final_dataset, fpc_combos_list)

        # Combines the final_datasets together
        full_dataset = final_dataset_1 + final_dataset_2 + final_dataset_3 + final_dataset_4 + final_dataset_5 + final_dataset_6 + final_dataset_7 + final_dataset_8

        # Shuffles the combined set
        shuffle(full_dataset)

        dataset = []

        # Loops through the full dataset and removes any duplicates and any excess files
        for file_paths in full_dataset:
          if file_paths not in dataset:
            dataset.append(file_paths)

        return dataset

      elif true_g == 'F':
        #Creates first set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='A', participant_gender='F')
        final_dataset_1 = create_final_dataset(pre_final_dataset, fpc_combos_list)

        #Creates second set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='B', participant_gender='F')
        final_dataset_2 = create_final_dataset(pre_final_dataset, fpc_combos_list)

        #Creates third set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='H', participant_gender='F')
        final_dataset_3 = create_final_dataset(pre_final_dataset, fpc_combos_list)

        #Creates fourth set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='W', participant_gender='F')
        final_dataset_4 = create_final_dataset(pre_final_dataset, fpc_combos_list)

        #Creates fith set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='A', participant_gender='F')
        final_dataset_5 = create_final_dataset(pre_final_dataset, fpc_combos_list)

        #Creates sixth set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='B', participant_gender='F')
        final_dataset_6 = create_final_dataset(pre_final_dataset, fpc_combos_list)

        #Creates seventh set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='H', participant_gender='F')
        final_dataset_7 = create_final_dataset(pre_final_dataset, fpc_combos_list)

        #Creates eigth set
        file_paths_list, file_names_list = file_path_list_generator(directory)
        split_file_names_list = split_file_names(file_names_list)
        pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='W', participant_gender='F')
        final_dataset_8 = create_final_dataset(pre_final_dataset, fpc_combos_list)
        
        # Combines the final_datasets together
        full_dataset = final_dataset_1 + final_dataset_2 + final_dataset_3 + final_dataset_4 + final_dataset_5 + final_dataset_6 + final_dataset_7 + final_dataset_8
     
        # Shuffles the combined set
        shuffle(full_dataset)

        dataset = []

        # Loops through the full dataset and removes any duplicates and any excess files
        for file_paths in full_dataset:
          if file_paths not in dataset:
            dataset.append(file_paths)

        return dataset


In [ ]:
def both_are_other(directory):
      #Creates first set
      file_paths_list, file_names_list = file_path_list_generator(directory)
      split_file_names_list = split_file_names(file_names_list)
      pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='A', participant_gender='M')
      final_dataset_1 = create_final_dataset(pre_final_dataset, fpc_combos_list)

      #Creates second set
      file_paths_list, file_names_list = file_path_list_generator(directory)
      split_file_names_list = split_file_names(file_names_list)
      pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='B', participant_gender='M')
      final_dataset_2 = create_final_dataset(pre_final_dataset, fpc_combos_list)

      #Creates third set
      file_paths_list, file_names_list = file_path_list_generator(directory)
      split_file_names_list = split_file_names(file_names_list)
      pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='H', participant_gender='M')
      final_dataset_3 = create_final_dataset(pre_final_dataset, fpc_combos_list)

      #Creates fourth set
      file_paths_list, file_names_list = file_path_list_generator(directory)
      split_file_names_list = split_file_names(file_names_list)
      pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='W', participant_gender='M')
      final_dataset_4 = create_final_dataset(pre_final_dataset, fpc_combos_list)

      #Creates fith set
      file_paths_list, file_names_list = file_path_list_generator(directory)
      split_file_names_list = split_file_names(file_names_list)
      pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='A', participant_gender='F')
      final_dataset_5 = create_final_dataset(pre_final_dataset, fpc_combos_list)

      #Creates sixth set
      file_paths_list, file_names_list = file_path_list_generator(directory)
      split_file_names_list = split_file_names(file_names_list)
      pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='B', participant_gender='F')
      final_dataset_6 = create_final_dataset(pre_final_dataset, fpc_combos_list)

      #Creates seventh set
      file_paths_list, file_names_list = file_path_list_generator(directory)
      split_file_names_list = split_file_names(file_names_list)
      pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='H', participant_gender='F')
      final_dataset_7 = create_final_dataset(pre_final_dataset, fpc_combos_list)

      #Creates eigth set
      file_paths_list, file_names_list = file_path_list_generator(directory)
      split_file_names_list = split_file_names(file_names_list)
      pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race='W', participant_gender='F')
      final_dataset_8 = create_final_dataset(pre_final_dataset, fpc_combos_list)

      # Combines the final_datasets together
      full_dataset = final_dataset_1 + final_dataset_2 + final_dataset_3 + final_dataset_4 + final_dataset_5 + final_dataset_6 + final_dataset_7 + final_dataset_8

      # Shuffles the combined set
      shuffle(full_dataset)

      dataset = []

      # Loops through the full dataset and removes any duplicates and any excess files
      for file_paths in full_dataset:
        if file_paths not in dataset:
          dataset.append(file_paths)

      return dataset


In [ ]:
def dataset(directory, participant_race, participant_gender):
    '''
    This is going to combine 'final datasets' together, shuffle it, and returns a dataset
    ''' 


    if participant_gender == 'O' and participant_race != 'O':
      true_r = participant_race
      dataset = only_gender_is_other(directory, true_r)
      return dataset


    
    elif participant_race == 'O' and participant_gender != 'O':
      true_g = participant_race
      dataset = only_race_is_other(directory, true_g)
      return dataset



    elif participant_gender == 'O' and participant_race == 'O':
      dataset = both_are_other(directory)
      return dataset

    
    else:
      #Creates first set
      file_paths_list, file_names_list = file_path_list_generator(directory)
      split_file_names_list = split_file_names(file_names_list)
      pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race, participant_gender)
      final_dataset_1 = create_final_dataset(pre_final_dataset, fpc_combos_list)

      #Creates second set
      file_paths_list, file_names_list = file_path_list_generator(directory)
      split_file_names_list = split_file_names(file_names_list)
      pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race, participant_gender)
      final_dataset_2 = create_final_dataset(pre_final_dataset, fpc_combos_list)

      #Creates third set
      file_paths_list, file_names_list = file_path_list_generator(directory)
      split_file_names_list = split_file_names(file_names_list)
      pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race, participant_gender)
      final_dataset_3 = create_final_dataset(pre_final_dataset, fpc_combos_list)

      #Creates fourth set
      file_paths_list, file_names_list = file_path_list_generator(directory)
      split_file_names_list = split_file_names(file_names_list)
      pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race, participant_gender)
      final_dataset_4 = create_final_dataset(pre_final_dataset, fpc_combos_list)

      #Creates fith set
      file_paths_list, file_names_list = file_path_list_generator(directory)
      split_file_names_list = split_file_names(file_names_list)
      pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race, participant_gender)
      final_dataset_5 = create_final_dataset(pre_final_dataset, fpc_combos_list)

      #Creates sixth set
      file_paths_list, file_names_list = file_path_list_generator(directory)
      split_file_names_list = split_file_names(file_names_list)
      pre_final_dataset = create_pre_final_dataset(file_paths_list, split_file_names_list, participant_race, participant_gender)
      final_dataset_6 = create_final_dataset(pre_final_dataset, fpc_combos_list)

      # Combines the final_datasets together
      full_dataset = final_dataset_1 + final_dataset_2 + final_dataset_3 + final_dataset_4 + final_dataset_5 + final_dataset_6

      # Shuffles the combined set
      shuffle(full_dataset)

      dataset = []

      # Loops through the full dataset and removes any duplicates and any excess files
      for file_paths in full_dataset:
        if file_paths not in dataset:
          dataset.append(file_paths)

      return dataset

In [ ]:
def condition_checker(ls):
  i = 0

  conditions = []
  condition = 'Bad'
  licycle = cycle(ls)

  # Prime the pump
  nextelem = next(licycle)

  while i <= len(ls) - 1:
      thiselem, nextelem = nextelem, next(licycle)

      conditions.append(thiselem[1][0] == nextelem[1][0] or thiselem[2][0] == nextelem[2][0] or thiselem[4] == nextelem[4])
      i += 1
  return conditions

In [ ]:
def generate_non_repeat(ds, participant_race, participant_gender):
  file_names_list = []

  for file_path in ds:
    file_name = file_path.replace(directory, '')
    file_names_list.append(file_name) 

  file_names_list = split_file_names(file_names_list)

  ls = file_names_list



  conditions = condition_checker(ls)
  last_list_hopefully = []

  i=0

  while i < len(ls):
    if conditions[i] == False:
        last_list_hopefully.append(ls[i])
    i+=1

  conditions = condition_checker(last_list_hopefully)
  determiner = True

  for condition in conditions[:20]:
    if condition == True:
      determiner = False
      break
    else: 
      determiner = True
      last_list_hopefully = last_list_hopefully[:19]
  
  return determiner, last_list_hopefully


In [ ]:
def tls_generator(directory, participant_race, participant_gender):
  determiner = False
  true_g = participant_gender 

  if participant_race == 'O' and participant_gender != 'O':
    while determiner == False:
      red = only_race_is_other(directory, true_g)
      determiner, llh = generate_non_repeat(red, 'O', true_g)

  else:
    while determiner == False:
      ds = dataset(directory, participant_race, participant_gender)
      determiner, llh = generate_non_repeat(ds, participant_race, participant_gender)

  llh = llh


  llh_to_xlsx = []

  for f_name in llh:
    llh_to_xlsx.append(('_'.join(f_name)) + '.jpg')

  
  df = pd.DataFrame()
    
  # Creating two columns
  df['file_path'] = llh_to_xlsx

  # Converting to excel

  return df.to_excel(fd + f'{participant_race}{participant_gender}.xlsx', index = False)

In [ ]:
tls_generator(directory, participant_race='B', participant_gender='F')


